In [9]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as functional

from PIL import Image
from torch.utils.data import Dataset, DataLoader, TensorDataset

In [10]:
def read_image(image_path):
    image = Image.open(image_path)
    image_data = np.asarray(image)
    return image_data

In [11]:
np.random.seed(6)

fig_size = (32, 32)

choice = [i * 7 % 68 for i in range(1, 25 + 1)]
choice = [0] + choice # Append the selfish part
pic_root_folder = "PIE"
selfish_folder = "0"
train_set_name, test_set_name = [], []
for i in choice:
    images = [f'{i}/{filename}' for filename in os.listdir(f'{pic_root_folder}/{i}')]
    np.random.shuffle(images)
    train_size = int(len(images) * 0.7)
    train_set_name += images[:train_size]
    test_set_name += images[train_size:]

# Generate the label for each set
train_set_label = np.array([int(i.split('/')[0]) for i in train_set_name])
test_set_label = np.array([int(i.split('/')[0]) for i in test_set_name])

# Read and transform image data
train_set_data = np.zeros((len(train_set_name), *fig_size))
test_set_data = np.zeros((len(test_set_name), *fig_size))
for i in range(len(train_set_name)):
    train_set_data[i, :] = read_image(f'{pic_root_folder}/{train_set_name[i]}')
for i in range(len(test_set_name)):
    test_set_data[i, :] = read_image(f'{pic_root_folder}/{test_set_name[i]}')

In [12]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, kernel_size=5)
        self.conv2 = nn.Conv2d(20, 50, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(50 * 5 * 5, 500)
        self.fc2 = nn.Linear(500, 26)

    def forward(self, x):
        x = self.pool(self.conv1(x))
        x = self.pool(self.conv2(x))
        x = x.view(-1, 50 * 5 * 5)
        x = self.fc2(self.fc1(x))
        x = functional.relu(x)
        return x

In [13]:
unique_labels = np.unique(train_set_label)
label_to_index = {unique_labels[i]: i for i in range(26)}
index_to_lable = {i:unique_labels[i] for i in range(26)}

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

X_train = train_set_data[:, None, :, :] / 255.0
Y_train = np.vectorize(label_to_index.get)(train_set_label)
X_train_tensor = torch.tensor(X_train).float().to(device)
y_train_tensor = torch.tensor(Y_train).long().to(device)
tensor_dataset = TensorDataset(X_train_tensor, y_train_tensor)
dataset_loader = DataLoader(dataset=tensor_dataset, batch_size=64, shuffle=True)

Using device: cuda


In [15]:
model = CNN().to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
epoch = 15

for i in range(epoch):
    total_loss = 0.0
    correct_predictions, total_predictions = 0, 0
    for images, labels in dataset_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        output = model(images)
        loss = loss_function(output, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(output.data, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

    loss = total_loss / len(dataset_loader)
    accuracy = correct_predictions / total_predictions * 100
    print(f'Epoch {i + 1:>2}, Loss: {loss:.2f}, Accurcy: {accuracy:.2f}%')

Epoch  1, Loss: 3.18, Accurcy: 8.45%
Epoch  2, Loss: 1.62, Accurcy: 58.30%
Epoch  3, Loss: 0.46, Accurcy: 88.06%
Epoch  4, Loss: 0.22, Accurcy: 94.72%
Epoch  5, Loss: 0.20, Accurcy: 94.62%
Epoch  6, Loss: 0.10, Accurcy: 97.57%
Epoch  7, Loss: 0.07, Accurcy: 98.51%
Epoch  8, Loss: 0.04, Accurcy: 99.32%
Epoch  9, Loss: 0.03, Accurcy: 99.63%
Epoch 10, Loss: 0.02, Accurcy: 99.59%
Epoch 11, Loss: 0.02, Accurcy: 99.70%
Epoch 12, Loss: 0.01, Accurcy: 99.80%
Epoch 13, Loss: 0.01, Accurcy: 99.93%
Epoch 14, Loss: 0.01, Accurcy: 99.97%
Epoch 15, Loss: 0.01, Accurcy: 99.97%


In [16]:
X_test = test_set_data[:,None,:,:] / 255.0
X_test_tensor = torch.tensor(X_test).float().to(device)
model.eval()
with torch.no_grad():
    ouput = model(X_test_tensor)
    _, result = torch.max(ouput, 1)
    result = result.cpu()

predict_result = np.vectorize(index_to_lable.get)(result.numpy())
accuracy = np.sum(predict_result == test_set_label) / test_set_label.shape[0]
print(f'The accuracy on the testing set: {accuracy * 100:.2f}%')

The accuracy on the testing set: 98.70%
